In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [4]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0,converters={'body': eval})
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [6]:
len(labels)

78

In [7]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 300


In [8]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [9]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [10]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [11]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [12]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12935, 300) 
test:  (4312, 300) 
val:  (4312, 300) 
y_tain: (12935, 78)


In [13]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [14]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [15]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [16]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [17]:
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn

    cnn_1 = Conv2D(1,name='cnn_1', 
                 kernel_size=(5, sentence_maxlen),
                 activation='relu')(embd_sentence)

    cnn_2 =  Lambda(lambda x: K.sum(x, axis=3),name='cnn_2')(cnn_1)
    max_pool_1 = MaxPooling1D(3,name='max_pool_1')(cnn_2)
    # bn_1 = BatchNormalization(name='bn_1')(max_pool_1)
    cnn_3 = Lambda(lambda x: K.sum(x, axis=2),name='cnn_3')(max_pool_1)
    # bn_2 = BatchNormalization(name='bn_2')(cnn_3)

    flatten = Flatten(name='flatten')(cnn_3)
    dense_1 = Dense(400,activation="relu",name='dense_1')(flatten)
    dense_2 = Dense(300,activation="relu",name='dense_2')(dense_1)
    dense_3 = Dense(200,activation="relu",name='dense_3')(dense_2)
    dense_4 = Dense(len(labels),activation="sigmoid",name='dense_4')(dense_3)
    out = Softmax(name='Soft_max_out')(dense_4)
    
    

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=["accuracy",
    "categorical_accuracy",]) 
    return model


model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())


NameError: ignored

In [ ]:
# model.fit(X_train, y_train,
#             batch_size=32,
#             epochs=5,
#             validation_data=(X_val, y_val))

In [ ]:
# model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5) # Model stop training after 50 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=20,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss


Epoch 1/20
405/405 [==============================] - 57s 139ms/step - loss: 0.1156 - accuracy: 0.0102 - categorical_accuracy: 0.0102 - val_loss: 0.1130 - val_accuracy: 0.0141 - val_categorical_accuracy: 0.0141

Epoch 00001: val_loss improved from inf to 0.11301, saving model to best_cnn_4fc.h5
Epoch 2/20
405/405 [==============================] - 55s 137ms/step - loss: 0.1145 - accuracy: 0.0107 - categorical_accuracy: 0.0107 - val_loss: 0.1129 - val_accuracy: 0.0141 - val_categorical_accuracy: 0.0141

Epoch 00002: val_loss improved from 0.11301 to 0.11289, saving model to best_cnn_4fc.h5
Epoch 3/20
405/405 [==============================] - 57s 140ms/step - loss: 0.1146 - accuracy: 0.0117 - categorical_accuracy: 0.0117 - val_loss: 0.1127 - val_accuracy: 0.0135 - val_categorical_accuracy: 0.0135

Epoch 00003: val_loss improved from 0.11289 to 0.11275, saving model to best_cnn_4fc.h5
Epoch 4/20
405/405 [==============================] - 56s 138ms/step - loss: 0.1143 - accuracy: 0.0097 -

In [20]:
# model.save('CNN_2_no_binary.h5')

from keras.models import load_model
model = load_model('CNN_2_no_binary.h5')

Evaluation

In [21]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [22]:
print(pred[0])#example

[0.00891698 0.00979597 0.00865234 0.00858508 0.01216903 0.00858476
 0.00861608 0.00858577 0.00860617 0.00864381 0.0233189  0.00947236
 0.00861769 0.00868051 0.00858477 0.01684724 0.00876226 0.02206947
 0.02259556 0.00873951 0.01260837 0.01190028 0.00858474 0.02065245
 0.00934172 0.02308717 0.00928295 0.00943884 0.00888762 0.009358
 0.01175373 0.01756092 0.0216513  0.0233357  0.01098049 0.02270186
 0.0120183  0.00985914 0.00901982 0.00859309 0.01979674 0.00875096
 0.00858674 0.01914104 0.01093978 0.00858741 0.00858576 0.00992736
 0.00955887 0.00894604 0.00858808 0.00858498 0.0086178  0.01125179
 0.00959706 0.01899136 0.0209897  0.00915571 0.02319692 0.01829348
 0.01900753 0.00859344 0.01453963 0.00867797 0.00873454 0.02006797
 0.00858477 0.0103243  0.01525238 0.00959792 0.02189832 0.02314141
 0.012448   0.00858718 0.00858501 0.02300325 0.0106986  0.0187095 ]


In [46]:
y_pred=[]
for l in pred:
  temp=[]
  for value in l:
    if value>= np.mean(pred[0]) + .75*np.sqrt(np.var(pred[0])):
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [47]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.07      0.64      0.12       122
           1       0.06      0.64      0.12       139
           2       0.09      0.78      0.15       134
           3       0.00      0.00      0.00        16
           4       0.39      0.89      0.54        73
           5       0.00      0.00      0.00        10
           6       0.12      0.72      0.20       137
           7       0.00      0.00      0.00        37
           8       0.00      0.00      0.00        33
           9       0.07      0.42      0.12       121
          10       0.04      1.00      0.07       164
          11       0.21      0.62      0.31        65
          12       0.50      0.99      0.67       140
          13       0.00      0.00      0.00        18
          14       0.00      0.00      0.00        12
          15       0.05      0.80      0.10       138
          16       0.15      0.74      0.25       124
          17       0.13    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
